In [1]:
from langchain_groq import ChatGroq

In [30]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

# Load variables from .env into environment
load_dotenv()

# Get GROQ API Key from env
groq_api_key = os.getenv("GROQ_API_KEY")

# Create LLM instance
llm = ChatGroq(
    temperature=0,
    groq_api_key=groq_api_key,
    model_name="llama3-70b-8192"  # Updated correct model name as per Groq docs
)

# Invoke the LLM
response = llm.invoke("The first person to land on the moon was ...")
print(response.content)

Neil Armstrong! On July 20, 1969, Neil Armstrong became the first person to set foot on the moon, famously declaring "That's one small step for man, one giant leap for mankind" as he stepped off the lunar module Eagle onto the moon's surface.


In [7]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/retail-associate-seas-cary-nc/job/R-60680")
page_data = loader.load().pop().page_content
print(page_data)






















Retail Associate, SEAS - Cary, NC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous 

In [8]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [9]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Retail Associate, SEAS',
 'experience': 'No experience required',
 'skills': ['Customer service',
  'Communication',
  'Teamwork',
  'Adaptability',
  'Basic math skills',
  'Ability to learn and train on latest products and technologies'],
 'description': 'We believe that if you have a body, you are an athlete, which is why when you join our team as a Retail Associate, you are referred to as an Athlete. Are you ready to embrace it? Let’s do it. Lace Up as a Nike Retail Associate. As a Nike Retail Associate, you’re the face of NIKE. Enjoy high-volume and a fast pace as your diverse experience and perspective helps guide customers in making the best decisions for them. You’ll work with your team to focus on customer service and get to the win the right way.'}

In [10]:
type(json_res)

dict

In [19]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [22]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [17]:
job = json_res
job = 'skills'

In [27]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}]]

In [26]:
job

{'role': 'Retail Associate, SEAS',
 'experience': 'No experience required',
 'skills': ['Customer service',
  'Communication',
  'Teamwork',
  'Adaptability',
  'Basic math skills',
  'Ability to learn and train on latest products and technologies'],
 'description': 'We believe that if you have a body, you are an athlete, which is why when you join our team as a Retail Associate, you are referred to as an Athlete. Are you ready to embrace it? Let’s do it. Lace Up as a Nike Retail Associate. As a Nike Retail Associate, you’re the face of NIKE. Enjoy high-volume and a fast pace as your diverse experience and perspective helps guide customers in making the best decisions for them. You’ll work with your team to focus on customer service and get to the win the right way.'}

In [25]:
job = json_res
job['skills']

['Customer service',
 'Communication',
 'Teamwork',
 'Adaptability',
 'Basic math skills',
 'Ability to learn and train on latest products and technologies']

In [29]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Kevin, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhance Customer Experience with AtliQ's Automated Solutions

Dear Hiring Manager,

I came across the job posting for a Retail Associate at Nike and was impressed by the company's commitment to providing exceptional customer service. As a Business Development Executive at AtliQ, I believe our AI and software consulting services can help Nike streamline its retail operations and enhance customer experience.

At AtliQ, we have a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise in Magento ([https://example.com/magento-portfolio](https://example.com/magento-portfolio)) can help Nike optimize its e-commerce platform, while our machine learning capabilities in Python ([https://example.com/ml-python-portfolio](https://example.com/ml-python-portfolio)) can enable personalized customer recommendations and predictive analytics.

Our Android portfolio ([htt